In [2]:
import numpy as np
import pandas as pd
import json
import os

pd.set_option("max_rows", 10)

In [22]:
# Reading Spotify web API credentials from settings.env hidden file

with open('settings.env') as f:
    env_vars = json.loads(f.read())

# Set environment variables
os.environ['SPOTIPY_CLIENT_ID'] = env_vars['SPOTIPY_CLIENT_ID']
os.environ['SPOTIPY_CLIENT_SECRET'] = env_vars['SPOTIPY_CLIENT_SECRET']

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

In [3]:
df = pd.read_csv('../web_scraping/weekly_hot100_1962-2020.csv')

In [4]:
# Let's use only tracks from year 2000 or later

df = df[df['year'] >= 2000]

In [5]:
df.reset_index(drop=True, inplace=True)

In [10]:
df

,year,artist,title
0,2000,SON BY FOUR,Purest Of Pain (A Puro Dolor)
1,2000,MARC ANTHONY,You Sang To Me
2,2000,FAITH HILL,Breathe
3,2000,JO DEE MESSINA,That's The Way
4,2000,TONI BRAXTON,Just Be A Man About It
...,...,...,...
106895,2020,HALSEY,You Should Be Sad
106896,2020,LIL BABY,Woah
106897,2020,TREVOR DANIEL,Falling
106898,2020,LIZZO,Truth Hurts


In [5]:
df[df.duplicated(['artist', 'title'])].shape

(94, 4)

In [12]:
df.drop_duplicates(subset=['artist', 'title'], inplace=True)

In [13]:
df.isnull().sum()

year      0
artist    0
title     0
dtype: int64

In [14]:
df.reset_index(drop=True, inplace=True)

In [15]:
df

,year,artist,title
0,2000,SON BY FOUR,Purest Of Pain (A Puro Dolor)
1,2000,MARC ANTHONY,You Sang To Me
2,2000,FAITH HILL,Breathe
3,2000,JO DEE MESSINA,That's The Way
4,2000,TONI BRAXTON,Just Be A Man About It
...,...,...,...
8590,2020,"MIGOS, YOUNG THUG & TRAVIS SCOTT",g n f (Give No Fxk)
8591,2020,A BOOGIE WIT DA HOODIE featuring YOUNG THUG,Might Not Give Up
8592,2020,A BOOGIE WIT DA HOODIE featuring DABABY,Stain
8593,2020,A BOOGIE WIT DA HOODIE,Thug Love


In [16]:
# Cleaning artist and title names to avoid 'NaN'

import re

title = df['title'].tolist()
title_match = ['\'',
               '\s\(.*$']
title = [re.sub('|'.join(title_match), '', i) for i in title]

artist = df['artist'].tolist()
artist_match = ['\sFeaturing.*$',
                '\sfeaturing.*$',
                '\sFeauring.*$',
                '\s\(Featuring.*$',
                '\s\(With.*$',
                '\s\(Duet.*$',
                '\sIntroducing.*$',
                '\s\"Fenderella\"',
                'The West Coast Rap All-Stars',
                '\sVs.*$',
                '\sX\s.*$',
                '\sx\s.*$',
                '\sWith\s.*$',
                '\swith\s.*$',
                '\sPresents.*$',
                '\sPresents.*$',
                '\s&\s.*$',
                ',.*$',
                '\'']

artist = [re.sub('|'.join(artist_match), '', i) for i in artist]

In [17]:
df['artist'] = artist
df['title'] = title

In [18]:
df

,year,artist,title
0,2000,SON BY FOUR,Purest Of Pain
1,2000,MARC ANTHONY,You Sang To Me
2,2000,FAITH HILL,Breathe
3,2000,JO DEE MESSINA,Thats The Way
4,2000,TONI BRAXTON,Just Be A Man About It
...,...,...,...
8590,2020,MIGOS,g n f
8591,2020,A BOOGIE WIT DA HOODIE,Might Not Give Up
8592,2020,A BOOGIE WIT DA HOODIE,Stain
8593,2020,A BOOGIE WIT DA HOODIE,Thug Love


In [23]:
track_id_list = []

for i in range(0,len(df['artist'])):
    search = sp.search(q='artist:' + df['artist'][i] + ' track:' + df['title'][i], type='track')
    if (search['tracks']['items'] == []):
         track_id_list.append(np.nan)
    else:
        track_id = search['tracks']['items'][0]['id']
        track_id_list.append(track_id)

In [24]:
df['track_id'] = track_id_list

In [25]:
df.isnull().sum()

year          0
artist        0
title         0
track_id    507
dtype: int64

In [26]:
df[df['track_id'].isnull()]

,year,artist,title,track_id
8,2000,PINK,Most Girls,NaN
18,2000,SOULDECISION,Faded,NaN
28,2000,BB MAK,Back Here,NaN
35,2000,LIL BOW WOW,Bounce With Me,NaN
54,2000,ERIC HEATHERLEY,Flowers In The Wall,NaN
...,...,...,...,...
8286,2020,JHENE AIKO,Pussy Fairy,NaN
8323,2020,SAM SMITH,Im Ready,NaN
8356,2020,KELSEY BALLERINI,the other girl,NaN
8494,2020,GARTH BROOKS,Dive Bar,NaN


In [27]:
# Removing rows with NaN in 'track_id'
df.dropna(inplace=True)

In [8]:
df[df['track_id'].duplicated()]

,year,artist,title,track_id
104,2000,NINE DAYS,Absolutely,3mNecsYFb6LQg7822DPXCP
107,2000,RED HOT CHILI PEPPERS,Californication,48UPSzbZjgc449aqz8bxox
109,2000,LIL ZANE,Callin Me,0io6VQqmCT8Yz1faAqBUFh
110,2000,MYA,Case Of The Ex,1ak0S3NhwWrUgNlQhJ1412
112,2000,CREED,With Arms Open Wide,0eKyHwckh9vQb8ncZ2DXCs
...,...,...,...,...
7878,2020,ARCANGEL,Sigues Con El,4Pu0a2TuHOYtI4CCE3HEXI
7999,2020,DJ SNAKE,Loco Contigo,6osaMSJh9NguagEDQcZaKx
8002,2020,DaBABY,Suge,2gwkD6igEhQbDQegRCcdoB
8015,2020,GUNNA,Skybox,7GwYENSg87oERcW0Wacd6m


In [9]:
# Remove duplicates
df.drop_duplicates(subset=['track_id'], inplace=True)

In [10]:
df.reset_index(drop=True, inplace=True)

In [11]:
df

,year,artist,title,track_id
0,2000,SON BY FOUR,Purest Of Pain,1MOqMyQ7CULmWWjovkFY5B
1,2000,MARC ANTHONY,You Sang To Me,2dwhMQsFeHr2S787WxqAqW
2,2000,FAITH HILL,Breathe,3y4LxiYMgDl4RethdzpmNe
3,2000,JO DEE MESSINA,Thats The Way,296XGtH5MeGisqD3uAz6Q6
4,2000,TONI BRAXTON,Just Be A Man About It,6kD36kVRn5leDDbjXpHQY0
...,...,...,...,...
7853,2020,MIGOS,g n f,6ZelF5APDN5r6XnFqcnvWR
7854,2020,A BOOGIE WIT DA HOODIE,Might Not Give Up,0bo4hZZV4aYI4EEh1IpDNa
7855,2020,A BOOGIE WIT DA HOODIE,Stain,5ipk9bW6E344oWoylOOm82
7856,2020,A BOOGIE WIT DA HOODIE,Thug Love,3bhvpwuvTXpoQMI3l41NQ0


In [12]:
df.to_csv('hot100.csv', encoding='utf-8', index=False)

In [3]:
df = pd.read_csv('hot100.csv')

In [4]:
df

,year,artist,title,track_id
0,2000,SON BY FOUR,Purest Of Pain,1MOqMyQ7CULmWWjovkFY5B
1,2000,MARC ANTHONY,You Sang To Me,2dwhMQsFeHr2S787WxqAqW
2,2000,FAITH HILL,Breathe,3y4LxiYMgDl4RethdzpmNe
3,2000,JO DEE MESSINA,Thats The Way,296XGtH5MeGisqD3uAz6Q6
4,2000,TONI BRAXTON,Just Be A Man About It,6kD36kVRn5leDDbjXpHQY0
...,...,...,...,...
8083,2020,MIGOS,g n f,6ZelF5APDN5r6XnFqcnvWR
8084,2020,A BOOGIE WIT DA HOODIE,Might Not Give Up,0bo4hZZV4aYI4EEh1IpDNa
8085,2020,A BOOGIE WIT DA HOODIE,Stain,5ipk9bW6E344oWoylOOm82
8086,2020,A BOOGIE WIT DA HOODIE,Thug Love,3bhvpwuvTXpoQMI3l41NQ0


In [33]:
# Get audio_features

audio = [sp.audio_features(x) for x in df['track_id']]

In [133]:
audio_df = pd.DataFrame()

empty_row = {'danceability': np.nan,
 'energy': np.nan,
 'key': np.nan,
 'loudness': np.nan,
 'mode': np.nan,
 'speechiness': np.nan,
 'acousticness': np.nan,
 'instrumentalness': np.nan,
 'liveness': np.nan,
 'valence': np.nan,
 'tempo': np.nan,
 'type': np.nan,
 'id': np.nan,
 'uri': np.nan,
 'track_href': np.nan,
 'analysis_url': np.nan,
 'duration_ms': np.nan,
 'time_signature': np.nan}

for i in range(0,len(df['track_id'])):
    if type(audio[i][0]) != type(None):
        audio_df = audio_df.append(pd.json_normalize(audio[i][0]))
    else:
        audio_df = audio_df.append(pd.json_normalize(empty_row))

In [134]:
audio_df.reset_index(drop=True, inplace=True)

In [135]:
audio_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.565,0.547,0.0,-7.722,1.0,0.0347,0.5790,0.000000,0.1940,0.252,75.018,audio_features,1MOqMyQ7CULmWWjovkFY5B,spotify:track:1MOqMyQ7CULmWWjovkFY5B,https://api.spotify.com/v1/tracks/1MOqMyQ7CULm...,https://api.spotify.com/v1/audio-analysis/1MOq...,209320.0,4.0
1,0.578,0.894,10.0,-5.420,1.0,0.0296,0.0103,0.000003,0.2160,0.741,165.980,audio_features,2dwhMQsFeHr2S787WxqAqW,spotify:track:2dwhMQsFeHr2S787WxqAqW,https://api.spotify.com/v1/tracks/2dwhMQsFeHr2...,https://api.spotify.com/v1/audio-analysis/2dwh...,347107.0,4.0
2,0.529,0.496,7.0,-9.007,1.0,0.0290,0.1730,0.000000,0.2510,0.278,136.859,audio_features,3y4LxiYMgDl4RethdzpmNe,spotify:track:3y4LxiYMgDl4RethdzpmNe,https://api.spotify.com/v1/tracks/3y4LxiYMgDl4...,https://api.spotify.com/v1/audio-analysis/3y4L...,250547.0,4.0
3,0.488,0.923,2.0,-3.697,1.0,0.1030,0.1290,0.000000,0.1580,0.818,183.891,audio_features,296XGtH5MeGisqD3uAz6Q6,spotify:track:296XGtH5MeGisqD3uAz6Q6,https://api.spotify.com/v1/tracks/296XGtH5MeGi...,https://api.spotify.com/v1/audio-analysis/296X...,202253.0,4.0
4,0.753,0.450,9.0,-6.909,1.0,0.0924,0.2740,0.000002,0.3210,0.560,109.405,audio_features,6kD36kVRn5leDDbjXpHQY0,spotify:track:6kD36kVRn5leDDbjXpHQY0,https://api.spotify.com/v1/tracks/6kD36kVRn5le...,https://api.spotify.com/v1/audio-analysis/6kD3...,288933.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8083,0.904,0.627,1.0,-5.608,1.0,0.2220,0.0303,0.000006,0.0923,0.236,145.976,audio_features,6ZelF5APDN5r6XnFqcnvWR,spotify:track:6ZelF5APDN5r6XnFqcnvWR,https://api.spotify.com/v1/tracks/6ZelF5APDN5r...,https://api.spotify.com/v1/audio-analysis/6Zel...,224243.0,4.0
8084,0.687,0.592,2.0,-6.378,1.0,0.2250,0.2030,0.000000,0.2790,0.245,178.830,audio_features,0bo4hZZV4aYI4EEh1IpDNa,spotify:track:0bo4hZZV4aYI4EEh1IpDNa,https://api.spotify.com/v1/tracks/0bo4hZZV4aYI...,https://api.spotify.com/v1/audio-analysis/0bo4...,225266.0,4.0
8085,0.816,0.754,8.0,-3.911,1.0,0.3880,0.0117,0.000000,0.1290,0.604,159.073,audio_features,5ipk9bW6E344oWoylOOm82,spotify:track:5ipk9bW6E344oWoylOOm82,https://api.spotify.com/v1/tracks/5ipk9bW6E344...,https://api.spotify.com/v1/audio-analysis/5ipk...,175128.0,4.0
8086,0.543,0.643,5.0,-4.707,0.0,0.2880,0.4270,0.000000,0.1740,0.326,178.105,audio_features,3bhvpwuvTXpoQMI3l41NQ0,spotify:track:3bhvpwuvTXpoQMI3l41NQ0,https://api.spotify.com/v1/tracks/3bhvpwuvTXpo...,https://api.spotify.com/v1/audio-analysis/3bhv...,181744.0,4.0


In [145]:
# Merging both dataframes

df_audio = df.merge(audio_df, left_index=True, right_index=True, how='outer')

In [147]:
df_audio['success'] = 1.0

In [148]:
df_audio

,year,artist,title,track_id,danceability,energy,key,loudness,mode,speechiness,...,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,success
0,2000,SON BY FOUR,Purest Of Pain,1MOqMyQ7CULmWWjovkFY5B,0.565,0.547,0.0,-7.722,1.0,0.0347,...,0.252,75.018,audio_features,1MOqMyQ7CULmWWjovkFY5B,spotify:track:1MOqMyQ7CULmWWjovkFY5B,https://api.spotify.com/v1/tracks/1MOqMyQ7CULm...,https://api.spotify.com/v1/audio-analysis/1MOq...,209320.0,4.0,1.0
1,2000,MARC ANTHONY,You Sang To Me,2dwhMQsFeHr2S787WxqAqW,0.578,0.894,10.0,-5.420,1.0,0.0296,...,0.741,165.980,audio_features,2dwhMQsFeHr2S787WxqAqW,spotify:track:2dwhMQsFeHr2S787WxqAqW,https://api.spotify.com/v1/tracks/2dwhMQsFeHr2...,https://api.spotify.com/v1/audio-analysis/2dwh...,347107.0,4.0,1.0
2,2000,FAITH HILL,Breathe,3y4LxiYMgDl4RethdzpmNe,0.529,0.496,7.0,-9.007,1.0,0.0290,...,0.278,136.859,audio_features,3y4LxiYMgDl4RethdzpmNe,spotify:track:3y4LxiYMgDl4RethdzpmNe,https://api.spotify.com/v1/tracks/3y4LxiYMgDl4...,https://api.spotify.com/v1/audio-analysis/3y4L...,250547.0,4.0,1.0
3,2000,JO DEE MESSINA,Thats The Way,296XGtH5MeGisqD3uAz6Q6,0.488,0.923,2.0,-3.697,1.0,0.1030,...,0.818,183.891,audio_features,296XGtH5MeGisqD3uAz6Q6,spotify:track:296XGtH5MeGisqD3uAz6Q6,https://api.spotify.com/v1/tracks/296XGtH5MeGi...,https://api.spotify.com/v1/audio-analysis/296X...,202253.0,4.0,1.0
4,2000,TONI BRAXTON,Just Be A Man About It,6kD36kVRn5leDDbjXpHQY0,0.753,0.450,9.0,-6.909,1.0,0.0924,...,0.560,109.405,audio_features,6kD36kVRn5leDDbjXpHQY0,spotify:track:6kD36kVRn5leDDbjXpHQY0,https://api.spotify.com/v1/tracks/6kD36kVRn5le...,https://api.spotify.com/v1/audio-analysis/6kD3...,288933.0,4.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8083,2020,MIGOS,g n f,6ZelF5APDN5r6XnFqcnvWR,0.904,0.627,1.0,-5.608,1.0,0.2220,...,0.236,145.976,audio_features,6ZelF5APDN5r6XnFqcnvWR,spotify:track:6ZelF5APDN5r6XnFqcnvWR,https://api.spotify.com/v1/tracks/6ZelF5APDN5r...,https://api.spotify.com/v1/audio-analysis/6Zel...,224243.0,4.0,1.0
8084,2020,A BOOGIE WIT DA HOODIE,Might Not Give Up,0bo4hZZV4aYI4EEh1IpDNa,0.687,0.592,2.0,-6.378,1.0,0.2250,...,0.245,178.830,audio_features,0bo4hZZV4aYI4EEh1IpDNa,spotify:track:0bo4hZZV4aYI4EEh1IpDNa,https://api.spotify.com/v1/tracks/0bo4hZZV4aYI...,https://api.spotify.com/v1/audio-analysis/0bo4...,225266.0,4.0,1.0
8085,2020,A BOOGIE WIT DA HOODIE,Stain,5ipk9bW6E344oWoylOOm82,0.816,0.754,8.0,-3.911,1.0,0.3880,...,0.604,159.073,audio_features,5ipk9bW6E344oWoylOOm82,spotify:track:5ipk9bW6E344oWoylOOm82,https://api.spotify.com/v1/tracks/5ipk9bW6E344...,https://api.spotify.com/v1/audio-analysis/5ipk...,175128.0,4.0,1.0
8086,2020,A BOOGIE WIT DA HOODIE,Thug Love,3bhvpwuvTXpoQMI3l41NQ0,0.543,0.643,5.0,-4.707,0.0,0.2880,...,0.326,178.105,audio_features,3bhvpwuvTXpoQMI3l41NQ0,spotify:track:3bhvpwuvTXpoQMI3l41NQ0,https://api.spotify.com/v1/tracks/3bhvpwuvTXpo...,https://api.spotify.com/v1/audio-analysis/3bhv...,181744.0,4.0,1.0


In [149]:
df_audio.to_csv('hot100_with_audio_features.csv', encoding='utf-8', index=False)